In [1]:
import re
import pandas as pd
from sqlalchemy import create_engine
import urllib

def load_file(filepath, columns):
    """Load specific columns from a CSV file."""
    return pd.read_csv(filepath, usecols=columns)

def validate_data(df, validation_rules):
    """Apply regex patterns or custom functions to validate data in dataframe columns."""
    for column, rule in validation_rules.items():
        if isinstance(rule, str):  # Regex pattern
            df[f'corrected_format_{column}'] = df[column].apply(
                lambda x: None if pd.isna(x) else (True if re.match(rule, str(x)) else False)
            )
        elif callable(rule):  # Function to handle complex validations
            df[f'corrected_format_{column}'] = df[column].apply(
                lambda x: None if pd.isna(x) else rule(x)
            )
    return df

def non_negative(value):
    """Return True if value is non-negative, else False."""
    try:
        return float(value) >= 0
    except ValueError:
        return False

def validate_date(value):
    """Validates date strings against the expected format 'Mon-YYYY', returning True if valid."""
    try:
        pd.to_datetime(value, format='%b-%Y')
        return True
    except ValueError:
        return False

# Updated summary function as described above
def compute_summary(df, column):
    """Print detailed summary for validation results of a specific column."""
    total_rows = df.shape[0]
    num_na = df[df[f'corrected_format_{column}'].isna()].shape[0]
    num_not_na = total_rows - num_na
    num_correct = df[(df[f'corrected_format_{column}'] == True) & (df[column].notna())].shape[0]
    num_incorrect = df[(df[f'corrected_format_{column}'] == False) & (df[column].notna())].shape[0]

    print(f'Total rows: {total_rows}')
    print(f'Number of rows with {column} is NULL: {num_na}')
    print(f'Number of rows with {column} is not NULL: {num_not_na}')
    print(f'Number of rows with correct {column} format and not null: {num_correct}')
    if num_not_na > 0:  # Prevent division by zero
        print(f'Percentage of rows with correct {column} format and not null: {num_correct / num_not_na * 100:.4f}%')
    else:
        print("No non-null data available to calculate percentage of correct format.")
    print(f'Number of rows with incorrect {column} format and not null: {num_incorrect}')
    if num_not_na > 0:  # Prevent division by zero
        print(f'Percentage of rows with incorrect {column} format and not null: {num_incorrect / num_not_na * 100:.4f}%')
    else:
        print("No non-null data available to calculate percentage of incorrect format.")
    print("-----------------------------------------------------------------------------------")

def save_to_sql(df, engine, table_name):
    # engine = create_engine(url)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Define your validation rules here
validation_rules = {
    'application_type': r'^(Individual|Joint App)$',
    'annual_inc': non_negative,
    'annual_inc_joint': r'^\d+(\.\d+)?$',
    'dti': lambda x: None if pd.isna(x) else (False if float(x) < 0 else True),
    'emp_length': r'^(< 1 year|1 year|2 years|3 years|4 years|5 years|6 years|7 years|8 years|9 years|10\+ years)$',
    'issue_d': validate_date,
    'int_rate': r'^\d+\.\d{2}%$',
    'home_ownership': r'^(ANY|MORTGAGE|NONE|OWN|RENT)$',
    'loan_status': r'^(Charged Off|Current|Default|Fully Paid|In Grace Period|Late \(16-30 days\)|Late \(31-120 days\))$',
    'loan_amnt': lambda x: None if pd.isna(x) else (False if not 1000 <= float(x) <= 40000 else True),
    'funded_amnt': lambda x: None if pd.isna(x) else (False if not 1000 <= float(x) <= 40000 else True),
    'installment': r'^\d+\.\d{2}$'
}

# Implement in main script
if __name__ == "__main__":
    filepath = 'LoanStats_web.csv'
    columns = list(validation_rules.keys())
    df = load_file(filepath, columns)
    df = validate_data(df, validation_rules)
    for column in validation_rules.keys():
        compute_summary(df, column)

    
    # Save to sql
    server = '34.125.58.101'
    database = 'TestDB'
    username = 'SA'
    password = 'Passw0rd123456'
    table_name = "loan_accuracy"
    
    # ตั้งค่าการเชื่อมต่อกับ MSSQL โดยใช้ข้อมูลที่เหมาะสมกับสภาพแวดล้อมของคุณ
    params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=server;DATABASE=database;UID=username;PWD=password")
    ##### engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
    # Using pymssql
    engine = create_engine(f'mssql+pymssql://{username}:{password}@{server}/{database}')
    
    save_to_sql(df, engine, table_name)



Total rows: 1432466
Number of rows with application_type is NULL: 26
Number of rows with application_type is not NULL: 1432440
Number of rows with correct application_type format and not null: 1432440
Percentage of rows with correct application_type format and not null: 100.0000%
Number of rows with incorrect application_type format and not null: 0
Percentage of rows with incorrect application_type format and not null: 0.0000%
-----------------------------------------------------------------------------------
Total rows: 1432466
Number of rows with annual_inc is NULL: 26
Number of rows with annual_inc is not NULL: 1432440
Number of rows with correct annual_inc format and not null: 1432440
Percentage of rows with correct annual_inc format and not null: 100.0000%
Number of rows with incorrect annual_inc format and not null: 0
Percentage of rows with incorrect annual_inc format and not null: 0.0000%
-----------------------------------------------------------------------------------
Total 

In [2]:
dev_df = pd.read_csv(filepath)[['application_type', 'annual_inc', 'annual_inc_joint', 'dti', 'dti_joint', 'emp_length'\
 ,'issue_d','int_rate','home_ownership','loan_status','loan_amnt', 'funded_amnt','installment']]

/var/folders/77/kb3ytnf93qd_ccggvdrcr2dw0000gn/T/ipykernel_47357/89215481.py:1: DtypeWarning: Columns (0,19,55,112,123,124,125,128,129,130,133,138,139,140) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_df = pd.read_csv(filepath)[['application_type', 'annual_inc', 'annual_inc_joint', 'dti', 'dti_joint', 'emp_length'\


In [3]:
dev_df.describe(include='all')

,application_type,annual_inc,annual_inc_joint,dti,dti_joint,emp_length,issue_d,int_rate,home_ownership,loan_status,loan_amnt,funded_amnt,installment
count,1432440,1.432440e+06,1.120830e+05,1.430856e+06,112081.000000,1323970,1432440,1432440,1432440,1432440,1.432440e+06,1.432440e+06,1.432440e+06
unique,2,NaN,NaN,NaN,NaN,11,30,258,5,7,NaN,NaN,NaN
top,Individual,NaN,NaN,NaN,NaN,10+ years,Mar-2017,11.49%,MORTGAGE,Current,NaN,NaN,NaN
freq,1320357,NaN,NaN,NaN,NaN,478304,148724,49032,701646,702223,NaN,NaN,NaN
mean,NaN,8.103458e+04,1.268432e+05,1.921187e+01,19.088683,NaN,NaN,NaN,NaN,NaN,1.537039e+04,1.537037e+04,4.566891e+02
std,NaN,1.341834e+05,7.798226e+04,1.597277e+01,7.843541,NaN,NaN,NaN,NaN,NaN,9.646026e+03,9.646028e+03,2.817140e+02
min,NaN,0.000000e+00,5.693510e+03,-1.000000e+00,0.000000,NaN,NaN,NaN,NaN,NaN,1.000000e+03,1.000000e+03,1.940000e+01
25%,NaN,4.800000e+04,8.500000e+04,1.199000e+01,13.350000,NaN,NaN,NaN,NaN,NaN,8.000000e+03,8.000000e+03,2.502900e+02
50%,NaN,6.800000e+04,1.130000e+05,1.800000e+01,18.640000,NaN,NaN,NaN,NaN,NaN,1.300000e+04,1.300000e+04,3.800300e+02
75%,NaN,9.666700e+04,1.500000e+05,2.483000e+01,24.420000,NaN,NaN,NaN,NaN,NaN,2.000000e+04,2.000000e+04,6.145400e+02


In [4]:
for n in dev_df.columns:
    print(dev_df[[n]].groupby(n).size())

application_type
Individual    1320357
Joint App      112083
dtype: int64
annual_inc
0.0           1553
1.0             39
3.0              1
8.0              1
10.0             2
              ... 
9550000.0        1
9573072.0        1
9757200.0        1
9930475.0        1
61000000.0       4
Length: 56320, dtype: int64
annual_inc_joint
5693.51       1
9000.00       1
11000.00      1
12500.00      1
13470.00      1
             ..
2000000.00    1
4200000.00    1
5653500.00    1
6282000.00    1
7874821.00    1
Length: 14933, dtype: int64
dti
-1.00         5
 0.00      1195
 0.01        11
 0.02        20
 0.03        11
           ... 
 962.12       1
 962.83       1
 994.40       1
 995.60       1
 999.00     114
Length: 10373, dtype: int64
dti_joint
0.00     14
0.03      1
0.11      3
0.12      1
0.13      1
         ..
55.52     1
61.28     4
61.90     4
63.66     1
69.49     1
Length: 4012, dtype: int64
emp_length
1 year        95926
10+ years    478304
2 years      130251
3 years  

In [5]:
dev_df.columns

Index(['application_type', 'annual_inc', 'annual_inc_joint', 'dti',
       'dti_joint', 'emp_length', 'issue_d', 'int_rate', 'home_ownership',
       'loan_status', 'loan_amnt', 'funded_amnt', 'installment'],
      dtype='object')

In [6]:
for n in dev_df.columns:
    print(n)

application_type
annual_inc
annual_inc_joint
dti
dti_joint
emp_length
issue_d
int_rate
home_ownership
loan_status
loan_amnt
funded_amnt
installment


In [7]:
dev_df[['application_type']].groupby('application_type').size()

application_type
Individual    1320357
Joint App      112083
dtype: int64